<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/%E3%83%AA%E3%82%AC%E3%83%B3%E3%83%89%E6%8E%A2%E7%B4%A2%E3%82%B3%E3%83%BC%E3%83%89_pubmed_1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install biopython

In [7]:
import pandas as pd
from Bio import Entrez
import time
import re

# BiopythonのEntrezモジュールを初期化
Entrez.email = "tarumi.wataru@gmail.com"  # あなたのメールアドレスを設定してください
Entrez.api_key = "d9702839c15b5365a594355c0c6974cc6207"  # あなたのAPIキーを設定してください

# エクセルファイルから遺伝子名を読み込む
excel_file = "/content/oocyte_sperm.xlsx"  # エクセルファイルのパスを指定してください
gene_df = pd.read_excel(excel_file, header=None)
ligand_gene_name = gene_df.iloc[0, 1]  # 一行目のリガンド情報から遺伝子名を取得
receptor_gene_name = gene_df.iloc[1, 1]  # 二行目の受容体情報から遺伝子名を取得

# 遺伝子名を正規表現パターンに変換
ligand_gene_pattern = re.compile(rf'\b{re.escape(ligand_gene_name)}\b', re.IGNORECASE)
receptor_gene_pattern = re.compile(rf'\b{re.escape(receptor_gene_name)}\b', re.IGNORECASE)

# 検索対象のキーワードを指定
search_keywords = [ligand_gene_name, receptor_gene_name]  # リガンドと受容体情報、およびキーワードを指定

# 遺伝子情報を取得し、指定したキーワードを含む論文の要約を検索
matching_genes = []

# リクエストの間隔を設定
request_interval = 5  # 5秒ごとにリクエストを送信

for gene_name in [ligand_gene_name, receptor_gene_name]:
    try:
        # PubMedで遺伝子名を使用して論文を検索
        search_query = f"{gene_name} AND {' AND '.join(search_keywords)}"
        handle = Entrez.esearch(db="pubmed", term=search_query, retmax=10)  # 検索結果の最大数を設定
        record = Entrez.read(handle)
        handle.close()

        # 検索結果からPubMed IDを取得
        pmids = record["IdList"]

        # PubMed IDを使用して論文の要約を取得し、遺伝子名が含まれるか確認
        for pmid in pmids:
            summary_handle = Entrez.esummary(db="pubmed", id=pmid)
            summary_record = Entrez.read(summary_handle)
            summary_handle.close()
            article_summary = summary_record[0].get("Title", "") + " " + summary_record[0].get("Abstract", "")

            # 要約に遺伝子名が含まれるか確認
            if ligand_gene_pattern.search(article_summary) or receptor_gene_pattern.search(article_summary):
                matching_genes.append((gene_name, pmid))

    except Exception as e:
        print(f"エラーが発生しました: {e}")

    # リクエストの間隔を設ける
    time.sleep(request_interval)

# 検索結果をCSVファイルに保存
if matching_genes:
    result_df = pd.DataFrame(matching_genes, columns=["Gene Name", "PubMed IDs"])
    result_csv = "matching_genes.csv"
    result_df.to_csv(result_csv, index=False)
    print(f"指定したキーワードを含む論文を持つ遺伝子は以下の通りです。結果は'{result_csv}'に保存されました:")
    print(result_df)
else:
    print("指定したキーワードを含む論文を持つ遺伝子は見つかりませんでした。")

指定したキーワードを含む論文を持つ遺伝子は見つかりませんでした。
